In [1]:
import pandas as pd
import numpy as np
import os
from data_reduction_columns import * 
import re


In [2]:
def convert_dates(data,column):

   # "Date" column in MMYYYY format
   # Convert the "Date" column to a string
   data[column] = data[column].astype(str)

   # Extract the month and year components
   data["{}_month".format(column)] = data[column].str[:-4].astype(int)
   data["{}_year".format(column)] = data[column].str[-4:].astype(int)

   return data


In [3]:
def map_property_state(data):
    data['property_state'].replace('Nan',0)
    data['property_state_n'] = pd.to_numeric(data['property_state'].map(STATES_TO_VAL))
    
    
    
    return data

In [4]:
def change_data_type(data):

   data = data.astype({'balance':'float','interest_rate':'float','loan_term':'int','insurance_percentage':'float'})

   return data



In [5]:
def calculate_loan_age(data):
   report_period_dt = pd.to_datetime(data['reporting_period'])
   origination_dt =  pd.to_datetime(data['origination_date'],format='%m%Y')
   loan_age = (report_period_dt.dt.year - origination_dt.dt.year) * 12 + (report_period_dt.dt.month - origination_dt.dt.month)
   data['loan_age'] = loan_age
   return data 

In [6]:
def change_loan_term(data):
   data['loan_term'] = (data['loan_term']/ 12).astype(int)   
   return data

In [7]:
# Define the chunk size (number of rows to read at a time)
chunk_size = 100000  # Adjust this based on your available memory
first_pass = True
def read_data(file_list):
    first_pass = True
    for file_name in file_list:
        print('Reading File %s' %file_name)
        reader = pd.read_table(DATA_DIR+file_name, sep ="|", names = Original_Headers["LoanData"],chunksize=chunk_size)
        for row in reader:
            data_in = row[fields["LoanData"]]
            year = re.findall("([0-9]*)", file_name)[0]
            quater = re.findall("Q([0-9]*)", file_name)[0]
            data_in['acquisition_date'] = pd.to_numeric(year)+ pd.to_numeric(quater) /4      
            data_in['reporting_period'] = pd.to_datetime(row['reporting_period'],format='%m%Y')                    
                    
            # Concatenate the processed chunks into a single DataFrame
            if(first_pass):
                data = data_in
                first_pass = False
            else:
                data = pd.concat([data, data_in])
        data = convert_dates(data,'origination_date') 
        data = calculate_loan_age(data)
        #data = change_loan_term(data)
        data.sort_values(by=['loanId', 'reporting_period'], ascending=[True, False], inplace=True)    
        data = data.drop_duplicates(subset='loanId', keep='first') 
        
        
        
        
    return data

In [8]:
# Specify the directory path
directory_path = DATA_DIR

pattern = f"{directory_path}/*Q*.csv"

# List all files in the directory
file_list = os.listdir(directory_path)

data = read_data(file_list)

data = map_property_state(data)

data = change_data_type(data)

data['foreclosured'] = data['foreclosure_date'].notnull().astype(int)
data = calculate_loan_age(data)
print('Saving to %s' %(DATA_DIR+DATA_FILE))
#print(data.head())
data.to_csv(DATA_DIR_OUTPUT+DATA_FILE,index =False)

print('Finished reduction.  Total rows  = %s' %(data.shape[0]))

Reading File 2016Q1.csv


C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:9: DtypeWarning: Columns (40,78,105) have mixed types. Specify dtype option on import or set low_memory=False.
  for row in reader:
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['acquisition_date'] = pd.to_numeric(year)+ pd.to_numeric(quater) /4
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['

C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:9: DtypeWarning: Columns (105) have mixed types. Specify dtype option on import or set low_memory=False.
  for row in reader:
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['acquisition_date'] = pd.to_numeric(year)+ pd.to_numeric(quater) /4
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['report

C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:9: DtypeWarning: Columns (78,105) have mixed types. Specify dtype option on import or set low_memory=False.
  for row in reader:
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['acquisition_date'] = pd.to_numeric(year)+ pd.to_numeric(quater) /4
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['rep

C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:9: DtypeWarning: Columns (78,105) have mixed types. Specify dtype option on import or set low_memory=False.
  for row in reader:
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['acquisition_date'] = pd.to_numeric(year)+ pd.to_numeric(quater) /4
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['rep

...

C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['acquisition_date'] = pd.to_numeric(year)+ pd.to_numeric(quater) /4
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['reporting_period'] = pd.to_datetime(row['reporting_period'],format='%m%Y')
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:13: SettingWithCopyWarning: 
A value is trying to be set 

C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:9: DtypeWarning: Columns (78) have mixed types. Specify dtype option on import or set low_memory=False.
  for row in reader:
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['acquisition_date'] = pd.to_numeric(year)+ pd.to_numeric(quater) /4
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['reporti

C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:9: DtypeWarning: Columns (105) have mixed types. Specify dtype option on import or set low_memory=False.
  for row in reader:
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['acquisition_date'] = pd.to_numeric(year)+ pd.to_numeric(quater) /4
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['report

C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['acquisition_date'] = pd.to_numeric(year)+ pd.to_numeric(quater) /4
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in['reporting_period'] = pd.to_datetime(row['reporting_period'],format='%m%Y')
C:\Users\Pallavi\AppData\Local\Temp\ipykernel_3360\2670357713.py:13: SettingWithCopyWarning: 
A value is trying to be set 

Saving to ../data/raw/reduced_data.csv
Finished reduction.  Total rows  = 4336086
